In [5]:
from bs4 import BeautifulSoup
import bs4
from natasha import NamesExtractor
from collections import defaultdict
import requests
import re
import pickle
import time
import json
from nltk import flatten

In [6]:
from collections import defaultdict

In [7]:
extractor = NamesExtractor()

### Собираем регулярки

##### Имена

Цветаева М <br>Л. Н. Толстой<br>И. А. Бунин<br>И.-В. Гёте<br>Э. Сетон-Томпсон<br>Андреев Л.<br>Куприн.<br>Паустовский<br>Сент-Экзюпери

In [19]:
# Первое - имя
# Второе - отчество
# Третье - фамилия

pat1 = '([А-Я][а-я]?\.)(?:(?: |\-)([А-Я][а-я]?\.))? ([А-Я]\w+(?:-\w+)?)'
pat2 = '[А-Я]\w+ [А-Я]\w+'
#pat3 = '([А-Я]\w+(?:-\w+)?) ([А-Я]\.)(?: ?([А-Я]\.))?'


name_pattern = re.compile('(?:' + pat1 + '|' + pat2 + ')')

#### Жанр

In [4]:
reg_wrt_type = ['Лирика', 'Сказки', 'Рассказы',
                'Стихотворения', 'Стихи', 'Новеллы',
                'Басни', 'Проза']


for index, i in enumerate(reg_wrt_type):
    ans = '(?:{}|{})'.format(i[0], i[0].lower()) + i[1:-1] + '(?:{}|[а-я])?'.format(i[-1])
    reg_wrt_type[index] = ans


reg_wrt_type = '|'.join(reg_wrt_type)
reg_wrt_type = re.compile(reg_wrt_type)

## Список из Википедии

In [5]:
def get_soup(url):
    
    '''
    Скачиваем страничку, делаем soup
    '''
    
    req = requests.get(url)
    if req.status_code == 200:
        html = req.text
        soup = BeautifulSoup(html, 'lxml')
        return soup
    return None

In [6]:

def get_names(pattern, soup):
    
    '''
    Собираем имена из содержания
    '''
    
    names = []
    for line in soup_wiki.findAll("span", {"class": "toctext"}):

        if line.text == 'Критика': break
  
        name = re.sub('&#160;', ' ', line.text)
        name = pattern.findall(line.text)#[0]
        if name != []: names.append(name[0])

    return names


In [ ]:
link_wiki = 'https://ru.wikisource.org/wiki/Викитека:Школьная_программа'
soup_wiki = get_soup(link_wiki)
wiki_names = get_names(name_pattern, soup_wiki)

In [50]:
with open('wiki_names.pickle', 'wb') as f:
    pickle.dump(wiki_names, f)

Работает плохо, из-за вложенной структуры данных. Переделаем, сохраняя структуру

# WIKI УЧЕБНИК 2

In [ ]:
def clean_line(line):
    
    """
    Убираем лишние символы
    """
    
    line = re.sub('\[править\]', '', line)
    line = re.sub('\xa0', ' ', line)
    return line

Загружаем данные

In [20]:
with open('wiki.html', 'r', encoding='utf-8') as f:
    html = f.read()
    soup = BeautifulSoup(html, 'lxml')

In [21]:
body = soup.find("div", {"class": "mw-content-ltr"})
body = list(body)[0]

Парсим все данные со страницы, сохряняя первоначальную вложенность данных: 

Русская литература --> 	Фольклор --> Устное народное творчество 
и тд

\+ сохраняем ссылки на тексты, чтобы потом их скачать

In [22]:
#  h2 = Русская литература
#  h3 = Фольклор
#  h4 = Устное народное творчество
#  ul = «Помню, я ещё младшенька была…»


base = 'ru.wikisource.org'
a = list(body.children)
h2, h3, h4 = False, False, False
d = {}


for i in body.children:

    if i.name == 'h2':
        h2 = clean_line(i.text)
        d[h2] = {}
    
    if i.name == 'h3':
        h3 = clean_line(i.text)
        if h2: d[h2][h3] = {}
    
    if i.name == 'h4':
        h4 = clean_line(i.text)
        if h3: d[h2][h3][h4] = []
        else: d[h2][h4] = []
            
    if i.name == 'ul':
        
        if h4 in d[h2]: place = d[h2][h4]
        if h3 in d[h2]:
            if h4 in d[h2][h3]: place = d[h2][h3][h4]
            else:
                if len(d[h2][h3]) == 0: d[h2][h3] = []
                place = d[h2][h3]
        else: 
            if len(d[h2]) == 0: d[h2] = []
            place = d[h2]

        for name in i.find_all('li'):
            
            if name.find('a') != None and 'title' in name.a.attrs:
                if '(страница не существует)' not in name.a['title']:
                    link = base + name.a['href']
            else: link = ''
            
            place.append((name.text, link))
    

In [23]:
d['Русская литература']['Литература XIX века'].keys()#['Поэты пушкинской поры']

dict_keys(['И. А. Крылов', 'Я. П. Полонский', 'В. А. Жуковский', 'А. С. Грибоедов', 'А. В. Кольцов', 'Е. А. Боратынский', 'К. Ф. Рылеев', 'К. Н. Батюшков', 'А. С. Пушкин', 'М. Ю. Лермонтов', 'Поэты пушкинской поры', 'Н. В. Гоголь', 'А. Н. Островский', 'И. С. Тургенев', 'Ф. И. Тютчев', 'А. А. Фет', 'А. Н. Майков', 'Л. Н. Толстой', 'Н. А. Некрасов', 'Н. М. Языков', 'Н. С. Лесков', 'Н. Г. Чернышевский', 'И. А. Гончаров', 'И. С. Никитин', 'А. Н. Толстой', 'А. К. Толстой', 'М. Е. Салтыков-Щедрин', 'Ф. М. Достоевский', 'В. Г. Короленко', 'А. П. Чехов', 'В. М. Гаршин'])

In [217]:
with open('/Users/Stoneberry/Desktop/школьный архив/name_lists/wiki_names.pickle', 'wb') as f:
    pickle.dump(d, f)

Создаем словарь только с "автором" и произведениями (= последние два уровня иерархии)

In [24]:
def name_splitter_wiki(d):

    for i in d:
        
        if isinstance(d[i], list):

            name_split = name_pattern.findall(i)
            s = sum(map(len, flatten(name_split)))
        
            if name_split != [] and s != 0: 
                line = name_split[0]
            else: line = i
            
            yield line, d[i]
          
        elif isinstance(d[i], dict): 
            yield from name_splitter_wiki(d[i])
        

In [25]:
names_wiki = {i[0]: i[1] for i in name_splitter_wiki(d)}

In [26]:
names_wiki['Устное народное творчество']

[('Помню, я ещё младшенька была…',
  'ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%BC%D0%BD%D1%8E,_%D1%8F_%D0%B5%D1%89%D1%91_%D0%BC%D0%BB%D0%B0%D0%B4%D1%88%D0%B5%D0%BD%D1%8C%D0%BA%D0%B0_%D0%B1%D1%8B%D0%BB%D0%B0'),
 ('Ай вы, ветры, ветры буйные…',
  'ru.wikisource.org/wiki/%D0%90%D0%B9_%D0%B2%D1%8B,_%D0%B2%D0%B5%D1%82%D1%80%D1%8B,_%D0%B2%D0%B5%D1%82%D1%80%D1%8B_%D0%B1%D1%83%D0%B9%D0%BD%D1%8B%D0%B5')]

In [27]:
names_wiki[('М.', 'В.', 'Ломоносов')]

[('Ода на день восшествия на престол Елисаветы Петровны',
  'ru.wikisource.org/wiki/%D0%9E%D0%B4%D0%B0_%D0%BD%D0%B0_%D0%B4%D0%B5%D0%BD%D1%8C_%D0%B2%D0%BE%D1%81%D1%88%D0%B5%D1%81%D1%82%D0%B2%D0%B8%D1%8F_%D0%BD%D0%B0_%D0%BF%D1%80%D0%B5%D1%81%D1%82%D0%BE%D0%BB_%D0%95%D0%BB%D0%B8%D1%81%D0%B0%D0%B2%D0%B5%D1%82%D1%8B_%D0%9F%D0%B5%D1%82%D1%80%D0%BE%D0%B2%D0%BD%D1%8B'),
 ('Вечернее размышление о Божием величестве при случае великого северного сияния',
  'ru.wikisource.org/wiki/%D0%92%D0%B5%D1%87%D0%B5%D1%80%D0%BD%D0%B5%D0%B5_%D1%80%D0%B0%D0%B7%D0%BC%D1%8B%D1%88%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BE_%D0%91%D0%BE%D0%B6%D0%B8%D0%B5%D0%BC_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B5_%D0%BF%D1%80%D0%B8_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B5_%D0%B2%D0%B5%D0%BB%D0%B8%D0%BA%D0%BE%D0%B3%D0%BE_%D1%81%D0%B5%D0%B2%D0%B5%D1%80%D0%BD%D0%BE%D0%B3%D0%BE_%D1%81%D0%B8%D1%8F%D0%BD%D0%B8%D1%8F'),
 ('Разговор с Анакреонтом',
  'ru.wikisource.org/wiki/%D0%A0%D0%B0%D0%B7%D0%B3%D0%BE%D0%B2%D0%BE%D1%80_%D

## Список из пдф файла - http://lickey3.narod.ru/spisok_reed.pdf

Файл содержит деление на классы, что полезно, сохраним эту структуру

In [229]:
def natasha_uni(line):
    
    '''
    Достаем имена с помощью Наташи
    Приводим к общему виду - А. С. Пушкин
    '''
    
    answer = []
   
    for match in extractor(line):
        if match.fact.last != None:
            
            ans = ''
            for index, item in enumerate(match.fact):
                if item != None:
                    if len(item) == 1: item += '.'
                    else: item = item.capitalize()  
                    ans += item + ' '

            answer.append(ans[:-1])

    return answer


def get_names_books_pdf(name_pattern, reg_wrt_type, line):
    
    '''
    Собираем имена и книги из каждой строки
    '''
    
    names = name_splitter_pdf(line)
    books = ''

    if names == []: names = natasha_uni(line)
            
    if len(names) == 1:
        
        books = re.findall('"(.+?)"', line)
                    
        if books == []:  # если ничего в кавычках нет, возможно, там написано через точку
            line = re.sub(names[0], '', line)
            books = line.split('.')
        if books == []: books = re.findall(reg_wrt_type, line)  # если и этого нет, ищем жанры
    
    return names, set(books)


Открываем файл

In [230]:
pdf_file = '/Users/Stoneberry/Desktop/школьный архив/spisok_reed.txt'

with open(pdf_file, 'r', encoding='utf-8') as f:
    pdf_file_lines = f.read()
    pdf_file_lines = re.sub('\.', '. ', pdf_file_lines)
    pdf_file_lines = re.sub('  ', ' ', pdf_file_lines)
    pdf_file_lines = re.sub('\x0c', '',  pdf_file_lines) 

In [ ]:
pdf_by_year = []  # [{'K': [1, 2, 3]}, {'L': [1, 2, 3]}]
pdf_by_names = defaultdict(set)  #{'K': {1, 2, 3}
pdf_summer_reading = {}  # 11 класс 
log = set()


for ind, item in enumerate(pdf_file_lines.split('\n\n')):
    
    lines = item.split('\n')
    if ind != 10: pdf_by_year.append({})
    
    for index, line in enumerate(lines):
        
        if index != 0:
            
            names, books = get_names_books_pdf(name_pattern, reg_wrt_type, line)

            if len(names) == 1:
                
                if books == []: log.add(line)
                elif ind == 10: pdf_summer_reading[names[0]] = books
                else:
                    pdf_by_year[-1][names[0]] = books
                    pdf_by_names[names[0]] |= books
               
            else: log.add(line)
  

In [37]:
pdf_by_year[0]

{'Л. Н. Толстой': {' Рассказы для детей'},
 'К. Д. Ушинский': {'Четыре желания'},
 'А. С. Пушкин': {'У лукоморья дуб зеленый'},
 'А. Л. Барто': {' Стихи для маленьких'},
 'К. И. Чуковский': {'Краденное солнце',
  'Мойдодыр',
  'Муха- Цокотуха',
  'Тараканище',
  'Телефон'},
 'В. Осеева': {' Рассказы'},
 'Б. С. Житков': {'Помощь идет'},
 'С. В. Михалков': {'А что у вас?', 'Дядя Степа'},
 'С. Я. Маршак': {'Вот такой рассеянный', 'Почта'},
 'В. В. Маяковский': {'Кем быть?',
  'Конь-огонь',
  'Что такое хорошо и что такое плохо'},
 'Н. Н. Носов': {' Рассказы'},
 'В. В. Бианки': {'Кто чем поет?', 'Чей нос лучше?'},
 'Ш. Пьеро': {'Кот в сапогах', 'Красная шапочка'}}

In [115]:
pdf_by_names['Л. Н. Толстой']

{' Рассказы для детей',
 'Анна Каренина',
 'Война и мир',
 'Воскресение',
 'Детство',
 'Золотой ключик или приключения Буратино',
 'Кавказский пленник',
 'Казаки',
 'Люцерн',
 'Набег',
 'Отрочество',
 'Севастопольские рассказы',
 'Утро помещика',
 'Хаджи-Мурат'}

In [116]:
pdf_summer_reading['А. И. Куприн']

{'',
 ' Гамбринус',
 ' Гранатовый браслет',
 ' Молох',
 ' Олеся',
 ' Поединок',
 ' Суламифь'}

В log попадают все случаи, когда в в строке несколько фамилий, или наташа не смогла там найти соответсвия. Про эту категорию подумаю позже

In [25]:
list(log)[:10]

['Братья Гримм Сказки',
 'Авторская песня: Высоцкий В. , Галич А. , Окуджава Б. , Цой В. , Тальков И. и др. ',
 'А. С. Пушкин. "Узник", "И. И. Пущину", "Зимнее утро", "Метель", "Туча", Станционный смотритель", "Ворон к ворону летит" Шотландская народная баллада',
 'Знакомство с серией книг "Жизнь замечательных людей"',
 'Хаггард. "Дочь Монтесумы", "Копи царя Соломона", "Прекрасная Маргарет"',
 'Гете "Фауст"',
 'Сказки разных народов',
 'Дефо "Робинзон Крузо". ',
 'Аверченко "Смерть африканского охотника"',
 'Свифт "Путешествия Гулливера" (можно в детском издании)']

In [48]:
with open('pdf_by_names.pickle', 'wb') as f:
    pickle.dump(pdf_by_names, f)

In [49]:
with open('pdf_by_year.pickle', 'wb') as f:
    pickle.dump(pdf_by_year, f)

Опять сделаем словарь "автор" - произведения

In [7]:
with open('pdf_by_names.pickle', 'rb') as f:
    pdf_names = pickle.load(f)   

In [9]:
def name_splitter_pdf(pdf_names):
    
    d = {}
    
    for i in pdf_names:
    
        rev_name_split = re.findall(pat3, i)
        s = sum(map(len, flatten(rev_name_split)))
    
        if rev_name_split != [] and s != 0: 
            line = rev_name_split[0]
            line = tuple(list(line[1:]) + [line[0]])
        
        else:
            name_split = name_pattern.findall(i)
            s = sum(map(len, flatten(name_split)))
            
            if name_split != [] and s != 0:
                line = name_split[0]
            else: line = i
      
        if line in d:
            d[line] |= pdf_names[i]
        else:
            d[line] = pdf_names[i]
        
    return d


In [13]:
pdf_names1 = name_splitter_pdf(pdf_names)

In [14]:
pdf_names1[('Л.', 'Н.', 'Толстой')]

{' Рассказы для детей',
 'Анна Каренина',
 'Война и мир',
 'Воскресение',
 'Детство',
 'Золотой ключик или приключения Буратино',
 'Кавказский пленник',
 'Казаки',
 'Люцерн',
 'Набег',
 'Отрочество',
 'Севастопольские рассказы',
 'Утро помещика',
 'Хаджи-Мурат'}

# Литрес

Тут произведения разделены по классам, но недостаточно четко, я решила, не учитывать это

In [15]:
with open('litrest.html', 'r', encoding='utf-8') as f:
    html = f.read()
    soup = BeautifulSoup(html, 'lxml')

In [16]:
links = {'5-6': 'https://www.litres.ru/kollekcii-knig/spisok-shkolnoy-literatury-5-6-klass/',
         '7-8': 'https://www.litres.ru/kollekcii-knig/spisok-shkolnoy-literatury-7-8-klass/',
         '9': 'https://www.litres.ru/kollekcii-knig/spisok-shkolnoy-literatury-9-klass/',
         '10-11': 'https://www.litres.ru/kollekcii-knig/spisok-shkolnoy-literatury-10-11-klass/'}

In [17]:
names_litres = {}


for link in links:
    
    for i in range(1, 6):
        
        l = links[link] + 'page-' + str(i)
    
        try:
            soup = get_soup(l)
            
            a = [line.text for line in soup.findAll("div", {"class": "art-item__author"})]
            b = [line.text for line in soup.findAll("div", {"class": "art-item__name"})]
    
            for index, item in enumerate(a):
                if item not in names_litres:
                    names_litres[item] = set()
                names_litres[item].add(b[index])
        
        except: continue


In [18]:
names_litres['Александр Пушкин']

{'Борис Годунов',
 'Дубровский',
 'Евгений Онегин',
 'Капитанская дочка',
 'Медный всадник',
 'Пиковая дама',
 'Полное собрание стихотворений',
 'Сказка о мертвой царевне и о семи богатырях',
 'Станционный смотритель'}

# Собираем все имена вместе

In [28]:
import pandas as pd

In [53]:
len(names_wiki), len(pdf_names1), len(names_litres)

(167, 144, 57)

In [112]:
def add_name(name, d):
    
    """
    Добавляем имя в таблицу
    """
    
    a = ['Имя', 'Отчество', 'Фамилия']
    
    if isinstance(name, tuple):
        for index, item in enumerate(name):
            d[a[index]].append(item)
       
    else:
        s = name.split(' ')
        
        if len(s) == 2:
            d['Имя'].append(s[0])
            d['Фамилия'].append(s[1])
            d['Отчество'].append('')
        
        else:
            d['Имя'].append('')
            d['Отчество'].append('')
            d['Фамилия'].append(name)
    

def add_to_table(book, d, link, w, p, l):
    
    """
    Добавляем информацию о тексте в таблицу
    """

    d['Название'].append(book)
    d['Ссылка'].append(link)
    d['Wiki'].append(w)
    d['PDF'].append(p)
    d['Litres'].append(l)


In [113]:
sources = [names_wiki, pdf_names1, names_litres]
s_index = {0:'Wiki', 1:'PDF', 2:'Litres'}

## Формируем сводную таблицу

In [114]:
d = {'Имя':[], 'Отчество':[], 'Фамилия':[], 'Название':[],
     'Ссылка':[], 'Wiki':[], 'PDF':[], 'Litres':[]}

hash_index = {}  # hash : номер строки в таблице 
cur_index = 0


for ind, source in enumerate(sources):
    
    for n in source:
    
        if isinstance(n, tuple):
            name = n[-1]
        else: name = n
    
        for b in source[n]:
            
            if ind == 0:
                link = b[1]
                book = b[0]
            else: book = b
      
            h = hash(book + name)
        
            if h not in hash_index:
                hash_index[h] = cur_index
                add_name(n, d)
                add_to_table(book, d, link, 0, 0, 0)
                index = cur_index
                cur_index += 1
            else:
                index = hash_index[h]
        
            d[s_index[ind]][index] = 1
        
        

In [172]:
data = pd.DataFrame(d)

In [173]:
data.head()

,Litres,PDF,Wiki,Имя,Название,Отчество,Ссылка,Фамилия
0,0,0,1,,"Помню, я ещё младшенька была…",,ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%BC%D0%B...,Устное народное творчество
1,0,0,1,,"Ай вы, ветры, ветры буйные…",,ru.wikisource.org/wiki/%D0%90%D0%B9_%D0%B2%D1%...,Устное народное творчество
2,0,0,1,Исторические,Ермак готовится к походу на Сибирь,,ru.wikisource.org/wiki/%D0%95%D1%80%D0%BC%D0%B...,песни
3,0,0,1,Исторические,Пугачёв в темнице,,ru.wikisource.org/wiki/%D0%9F%D1%83%D0%B3%D0%B...,песни
4,0,0,1,,Два Ивана — солдатских сына,,ru.wikisource.org/wiki/%D0%9D%D0%B0%D1%80%D0%B...,Русские народные сказки


In [174]:
data[data['Фамилия']=='Пушкин'].head()

,Litres,PDF,Wiki,Имя,Название,Отчество,Ссылка,Фамилия
64,0,0,1,А.,Повести Белкина\nМетель\nСтанционный смотрител...,С.,ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%B2%D0%B...,Пушкин
65,0,0,1,А.,Метель,С.,ru.wikisource.org/wiki/%D0%9C%D0%B5%D1%82%D0%B...,Пушкин
66,0,0,1,А.,Станционный смотритель,С.,ru.wikisource.org/wiki/%D0%A1%D1%82%D0%B0%D0%B...,Пушкин
67,0,0,1,А.,Выстрел,С.,ru.wikisource.org/wiki/%D0%92%D1%8B%D1%81%D1%8...,Пушкин
68,0,0,1,А.,Гробовщик,С.,ru.wikisource.org/wiki/%D0%93%D1%80%D0%BE%D0%B...,Пушкин


In [ ]:
data.to_csv('wiki_pdf_table.csv')

# Унифицируем имена

In [90]:
data.shape

(990, 8)

In [175]:
def name_shortner(line):
    
    """
    Сокращаем имена до первой буквы
    """
    
    if line != '':
        return line[0]
    return line


data['Short name'] = data['Имя'].apply(name_shortner)

In [176]:
data[data['Short name'] != ''].head()

,Litres,PDF,Wiki,Имя,Название,Отчество,Ссылка,Фамилия,Short name
2,0,0,1,Исторические,Ермак готовится к походу на Сибирь,,ru.wikisource.org/wiki/%D0%95%D1%80%D0%BC%D0%B...,песни,И
3,0,0,1,Исторические,Пугачёв в темнице,,ru.wikisource.org/wiki/%D0%9F%D1%83%D0%B3%D0%B...,песни,И
16,0,0,1,Древнерусская,Повесть временных лет (фрагменты),,ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%B2%D0%B...,литература,Д
17,0,0,1,Древнерусская,Слово о разорении Рязани Батыем (фрагменты),,ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%B2%D0%B...,литература,Д
18,0,0,1,Древнерусская,Житие Бориса и Глеба,,ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%B2%D0%B...,литература,Д


## Удаляем повторы

Опускаем, если у нас совпадает сокращенное имя, фамилия и название произведения. + Ставим 1 в тех полях оставленного вхождения, где еще встретился интересующий нас текст. 

In [177]:
dup_d = data[data.duplicated(subset=['Фамилия', 
                                     'Short name',
                                     'Название'], 
                             keep=False)]


for i in dup_d.groupby(['Фамилия', 'Short name', 'Название']):
    
    for l in i:
        
        if not isinstance(l, tuple):
            s = l[['Wiki', 'PDF', 'Litres']].sum(axis=0)
            
            for ind in l.index:
                for name in ['Wiki', 'PDF', 'Litres']:
                    data[name][ind] = s[name]


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [178]:
data_no_dub = data.drop_duplicates(subset=['Фамилия', 'Short name', 'Название'], keep='first', inplace=False)

In [125]:
data.shape

(990, 10)

In [179]:
data_no_dub.shape

(927, 9)

### Какие тексты есть сразу в трех документах: 

In [201]:
data_no_dub['Score'] = data_no_dub[['Litres', 'PDF', 'Wiki']].sum(axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [202]:
data_no_dub[data_no_dub['Score'] == 3]#.head()

,Litres,PDF,Wiki,Имя,Название,Отчество,Ссылка,Фамилия,Short name,Score
71,1,1,1,А.,Капитанская дочка,С.,ru.wikisource.org/wiki/%D0%9A%D0%B0%D0%BF%D0%B...,Пушкин,А,3
75,1,1,1,А.,Евгений Онегин,С.,ru.wikisource.org/wiki/%D0%95%D0%B2%D0%B3%D0%B...,Пушкин,А,3
76,1,1,1,А.,Борис Годунов,С.,ru.wikisource.org/wiki/%D0%91%D0%BE%D1%80%D0%B...,Пушкин,А,3
108,1,1,1,М.,Герой нашего времени,Ю.,ru.wikisource.org/wiki/%D0%93%D0%B5%D1%80%D0%B...,Лермонтов,М,3
111,1,1,1,М.,Мцыри,Ю.,ru.wikisource.org/wiki/%D0%9C%D1%86%D1%8B%D1%8...,Лермонтов,М,3
119,1,1,1,М.,Бородино,Ю.,ru.wikisource.org/wiki/%D0%91%D0%BE%D1%80%D0%B...,Лермонтов,М,3
143,1,1,1,А.,Гроза,Н.,ru.wikisource.org/wiki/%D0%93%D1%80%D0%BE%D0%B...,Островский,А,3
145,1,1,1,А.,Снегурочка,Н.,ru.wikisource.org/wiki/%D0%A1%D0%BD%D0%B5%D0%B...,Островский,А,3
147,1,1,1,И.,Отцы и дети,С.,ru.wikisource.org/wiki/%D0%9E%D1%82%D1%86%D1%8...,Тургенев,И,3
229,1,1,1,Н.,Левша,С.,ru.wikisource.org/wiki/%D0%9B%D0%B5%D0%B2%D1%8...,Лесков,Н,3


#### Coхраняем данные 

In [203]:
data_no_dub.to_csv('wiki_pdf_table_uni.csv')

# Находим года смерти авторов

Русские писатели, умершие раньше 1949 года

```
SELECT ?item ?itemLabel ?death WHERE {
  ?item wdt:P31 wd:Q5.
  ?item wdt:P172 wd:Q49542.
  ?item wdt:P106 wd:Q36180;
    wdt:P570 ?death.
  FILTER((YEAR(?death)) <= 1949 )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "ru". }
}
```

Люди, писавшие или говорившие на русском, умершие раньше 1949 года

```
SELECT ?item ?itemLabel ?death WHERE {
  ?item wdt:P31 wd:Q5.
  ?item wdt:P172 wd:Q49542;
    wdt:P570 ?death.
  FILTER((YEAR(?death)) <= 1949 )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "ru". }
}
```

In [40]:
with open('rus_lang_7k.json') as json_file:
    wikidata_7k = json.load(json_file)
    wikidata_7k = [i['itemLabel'] for i in wikidata_7k]

In [41]:
type(wikidata_7k)

list

In [43]:
wikidata_7k[:10]

['Екатерина II',
 'Фёдор Иванович Тютчев',
 'Саломе, Лу',
 'Николай II',
 'Константин Эдуардович Циолковский',
 'Иван Сергеевич Тургенев',
 'Иван Петрович Павлов',
 'Николай Васильевич Гоголь',
 'Афанасий',
 'Григорий Ефимович Распутин']

In [3]:
# Сергей Александрович Есенин
# Лу Синь 
# Канарис, Вильгельм Франц

wiki_data_surnames = set()

for line in wikidata_7k:
    
    if ',' in line: ans = line.split(',')[0]
    else: ans = line.split(' ')[-1]
    
    wiki_data_surnames.add(ans)

In [4]:
wiki_data_surnames

{'II',
 'Афанасий',
 'Гоголь',
 'Павлов',
 'Распутин',
 'Саломе',
 'Тургенев',
 'Тютчев',
 'Циолковский'}

In [52]:
len(wiki_data_surnames)

5337

In [60]:
len(yes), len(no)

(89, 81)

## План: 
    * Добавить ифнормацию о классе
    * Добавить год рождения человека